In [173]:
import pandas as pd
import numpy as np
import psycopg2
from analysys_funcs import *
import seaborn as sns
import matplotlib.pyplot as plt

In [154]:
# database connection

conn = psycopg2.connect(host='ep-black-pond-a2ydwdvs.eu-central-1.aws.neon.tech', database='Akademdb', user='Elizar54', password='XUpC1QOnGvA4')
cur = conn.cursor()

In [155]:
# getting data

df = pd.read_sql_query('select * from flats', conn)

C:\Users\eliza\AppData\Local\Temp\ipykernel_30676\3676301174.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('select * from flats', conn)


In [156]:
# general preproccessing

df['price'] = df['price'].apply(price_corr)
df.drop('id', inplace=True, axis=1)
df = df.drop_duplicates(keep=False)

for column in ['total_square', 'living_sq', 'kitchen_sq']:
    df[column] = df[column].apply(square_corr)

df['floor'] = df['floor'].apply(floor_corr)
df['ceil_height'] = df['ceil_height'].apply(lambda x: float(x.replace(',', '.').split()[0]) if x != 'None' else np.nan)
df['entrance'] = df['entrance'].apply(lambda x: int(x) if x != 'None' else np.nan)

df.drop(['warning', 'trash', 'barriers'], inplace=True, axis=1)

df = df.replace('None', np.nan)

df.dropna(thresh=7)

df['district'] = df['address'].apply(lambda x: x.split()[4])

df.drop('address', inplace=True, axis=1)

df['warm'].fillna(df['warm'].mode()[0], inplace=True)
df['home_type'].fillna(df['home_type'].mode()[0], inplace=True)
df['metro'] = df['metro'].isna().apply(lambda x: 'Yes' if x == False  else 'No')

In [165]:
# separating old and new flats

df_new = df[df['year'].isna() == False]
df_old = df[df['year'].isna() == True]

In [166]:
# working with nan values and feature engineering for old flats

df_old = df_old.dropna(subset=['living_sq', 'kitchen_sq'])
df_old.drop(['decor', 'year', 'floor'], axis=1, inplace=True)

df_old['elevators'] = df_old['elevators'].fillna(df_old['elevators'].mode()[0])
df_old['balcony'] = df_old['balcony'].fillna(df_old['balcony'].mode()[0])
df_old['toilets'] = df_old['toilets'].fillna(df_old['toilets'].mode()[0])
df_old['window_view'] = df_old['window_view'].fillna(df_old['window_view'].mode()[0])
df_old['ceil_height'] = df_old['ceil_height'].fillna(df_old['ceil_height'].mode()[0])
df_old['entrance'] = df_old['entrance'].fillna(df_old['entrance'].mode()[0])
df_old['ceil_height'] = df_old['ceil_height'].apply(lambda x: x if x != 0 else 2.7)

df_old['toilets_united'] = df_old['toilets'].apply(toilet_single)
df_old['toilets_sep'] = df_old['toilets'].apply(toilet_not_sngl)
df_old.drop('toilets', axis=1, inplace=True)

df_old['big_elev'] = df_old['elevators'].apply(big_elevators)
df_old['little_elev'] = df_old['elevators'].apply(lit_elevators)
df_old.drop('elevators', axis=1, inplace=True)

df_old['balcon'] = df_old['balcony'].apply(lambda x: int(x.split()[0]) if 'балкон' in x else 0)
df_old['lodgia'] = df_old['balcony'].apply(lambda x: int(x.split()[0]) if 'лоджия' in x else 0)
df_old.drop('balcony', axis=1, inplace=True)

df_old['warm'] = df_old['warm'].apply(lambda x: 'Центральное' if x == 'Нет информации' else x)

df_old_dummy = pd.get_dummies(df_old, columns=['rented', 'metro', 'window_view', 'home_type', 'warm', 'district'], dtype = 'float64')

In [167]:
df_new = df_new.dropna(subset=['living_sq', 'kitchen_sq'])
df_new.drop(['decor', 'year', 'floor', 'entrance', 'toilets', 'balcony', 'rented', 'window_view'], axis=1, inplace=True)

df_new['elevators'] = df_new['elevators'].fillna(df_new['elevators'].mode()[0])

df_new['ceil_height'] = df_new['ceil_height'].fillna(df_new['ceil_height'].mode()[0])

df_new['ceil_height'] = df_new['ceil_height'].apply(lambda x: x if x != 0 else 2.7)

df_new['big_elev'] = df_new['elevators'].apply(big_elevators)
df_new['little_elev'] = df_new['elevators'].apply(lit_elevators)
df_new.drop('elevators', axis=1, inplace=True)

df_new['warm'] = df_new['warm'].apply(lambda x: 'Центральное' if x == 'Нет информации' else x)

df_new_dummy = pd.get_dummies(df_new, columns=['home_type', 'warm', 'district', 'metro'], dtype = 'float64')

In [ ]:
# EDA (old flats)

plt.figure(figsize=(10, 6))

plt.xlabel('Площадь квартиры')
plt.ylabel('Цена квартиры')

sns.scatter(x=)
